In [6]:
import pandas as pd
import numpy as np
from nba_api.stats.endpoints import shotchartdetail, shotchartleaguewide, playercareerstats
from nba_api.stats.library.data import players


In [4]:
players_df = pd.DataFrame(data = players,
                         columns = ['id', 
                                    'first_name', 
                                    'last_name',
                                    'full_name',
                                    'is_active']).query('is_active == True')

In [11]:
import sqlite3

connection = sqlite3.connect("../Data/basketball.sqlite")

tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", connection)

def read_data(table, con=connection):
    return pd.read_sql(f"""SELECT * FROM {table}""", con)


In [14]:
tables.head()

,type,name,tbl_name,rootpage,sql
0,table,Player,Player,2,"CREATE TABLE ""Player"" (\n""id"" TEXT,\n ""full_n..."
1,table,Team,Team,4,"CREATE TABLE ""Team"" (\n""id"" TEXT,\n ""full_nam..."
2,table,Team_Attributes,Team_Attributes,5,"CREATE TABLE ""Team_Attributes"" (\n""ID"" TEXT,\n..."
3,table,Team_History,Team_History,6,"CREATE TABLE ""Team_History"" (\n""ID"" TEXT,\n ""..."
4,table,Player_Attributes,Player_Attributes,15,"CREATE TABLE ""Player_Attributes"" (\n""ID"" TEXT,..."


In [16]:
teams = read_data('Team')

In [83]:
from nba_api.stats.endpoints import commonteamroster
import time
from tqdm import tqdm

In [84]:
rosters = []

for team_id in tqdm(teams.id):
    rosters.append(commonteamroster.CommonTeamRoster(team_id, season='2020-21').get_data_frames()[0])
    time.sleep(0.5)

100%|████████████████████████████████████████████████████████████████| 30/30 [00:53<00:00,  1.78s/it]


In [85]:
roster_df = pd.concat(rosters)

In [87]:
roster_df.to_csv('roster2020-21.csv',index=False)

In [76]:
data = roster_df[['TeamID', 'PLAYER_ID']].values
shot_chart_details = []
for item in data:
    shot_chart_details.append(shotchartdetail.ShotChartDetail(team_id=item[0], 
                                                              player_id=item[1], 
                                                              season_nullable='2021-22').get_data_frames()[0])
    time.sleep(1)

In [79]:
shot_chart_details_2021_22 = pd.concat(shot_chart_details)
shot_chart_details_2021_22.to_csv('../Data/shot_chart_2021_22.csv', index=False)

In [98]:
def get_shot_chart_for_season(season_id='2021-22', teams=teams):
    rosters = []
    print('get roster')
    for team_id in tqdm(teams.id):
        rosters.append(commonteamroster.CommonTeamRoster(team_id, season=season_id).get_data_frames()[0])
        
    roster_df = pd.concat(rosters)
    roster_df.to_csv(f'../Data/roster{season_id}.csv',index=False)
    
    print('get_shot_chart deets')
    data = roster_df[['TeamID', 'PLAYER_ID']].values
    shot_chart_details = []
    for item in tqdm(data):
        shot_chart_details.append(shotchartdetail.ShotChartDetail(team_id=item[0], 
                                                                  player_id=item[1], 
                                                                  season_nullable=season_id,
                                                                 context_measure_simple='FGA').get_data_frames()[0])
        time.sleep(0.5)
        
    shot_chart_df = pd.concat(shot_chart_details)
    shot_chart_df.to_csv(f'../Data/shot_chart_{season_id}.csv', index=False)
    return shot_chart_df

In [99]:
shot_chart_details_2020_21 = get_shot_chart_for_season('2020-21')
shot_chart_details_2021_22 = get_shot_chart_for_season('2021-22')

get roster


100%|████████████████████████████████████████████████████████████████| 30/30 [00:25<00:00,  1.18it/s]


get_shot_chart deets


100%|██████████████████████████████████████████████████████████████| 503/503 [21:16<00:00,  2.54s/it]


get roster


100%|████████████████████████████████████████████████████████████████| 30/30 [00:33<00:00,  1.11s/it]


get_shot_chart deets


100%|██████████████████████████████████████████████████████████████| 508/508 [20:03<00:00,  2.37s/it]


In [91]:
df = shot_chart_details_2021_22

In [93]:
df.SHOT_ZONE_AREA.value_counts()

Center(C)                20229
Left Side Center(LC)      3275
Right Side Center(RC)     3186
Left Side(L)              2475
Right Side(R)             2434
Back Court(BC)               7
Name: SHOT_ZONE_AREA, dtype: int64

In [94]:
df.SHOT_ZONE_BASIC.value_counts()

Restricted Area          12955
Above the Break 3         7196
In The Paint (Non-RA)     5146
Mid-Range                 3739
Left Corner 3             1324
Right Corner 3            1241
Backcourt                    5
Name: SHOT_ZONE_BASIC, dtype: int64

In [97]:
df.groupby(['PLAYER_ID', 'PLAYER_NAME','SHOT_ZONE_AREA','SHOT_ZONE_BASIC']).count()

GRID_TYPE  GAME_ID  \
PLAYER_ID SHOT_ZONE_AREA       SHOT_ZONE_BASIC                             
2544      Center(C)            Above the Break 3             10       10   
                               In The Paint (Non-RA)          6        6   
                               Mid-Range                      3        3   
                               Restricted Area               73       73   
          Left Side Center(LC) Above the Break 3             18       18   
...                                                         ...      ...   
1630647   Left Side Center(LC) Above the Break 3              1        1   
1630688   Center(C)            Restricted Area                1        1   
1630691   Left Side Center(LC) Above the Break 3              1        1   
1630698   Center(C)            Restricted Area                1        1   
          Left Side Center(LC) Above the Break 3              2        2   

                                                      GAME_EVENT_ID  \
PLAYER_ID SHOT_ZONE_AREA       SHOT_ZONE_BASIC                        
2544      Center(C)            Above the Break 3                 10   
                               In The Paint (Non-RA)              6   
                               Mid-Range                          3   
                               Restricted Area                   73   
          Left Side Center(LC) Above the Break 3                 18   
...                                                             ...   
1630647   Left Side Center(LC) Above the Break 3                  1   
1630688   Center(C)            Restricted Area                    1   
1630691   Left Side Center(LC) Above the Break 3                  1   
1630698   Center(C)            Restricted Area                    1   
          Left Side Center(LC) Above the Break 3                  2   

                                                      PLAYER_NAME  TEAM_ID  \
PLAYER_ID SHOT_ZONE_AREA       SHOT_ZONE_BASIC                               
2544      Center(C)            Above the Break 3               10       10   
                               In The Paint (Non-RA)            6        6   
                               Mid-Range                        3        3   
                               Restricted Area                 73       73   
          Left Side Center(LC) Above the Break 3               18       18   
...                                                           ...      ...   
1630647   Left Side Center(LC) Above the Break 3                1        1   
1630688   Center(C)            Restricted Area                  1        1   
1630691   Left Side Center(LC) Above the Break 3                1        1   
1630698   Center(C)            Restricted Area                  1        1   
          Left Side Center(LC) Above the Break 3                2        2   

                                                      TEAM_NAME  PERIOD  \
PLAYER_ID SHOT_ZONE_AREA       SHOT_ZONE_BASIC                            
2544      Center(C)            Above the Break 3             10      10   
                               In The Paint (Non-RA)          6       6   
                               Mid-Range                      3       3   
                               Restricted Area               73      73   
          Left Side Center(LC) Above the Break 3             18      18   
...                                                         ...     ...   
1630647   Left Side Center(LC) Above the Break 3              1       1   
1630688   Center(C)            Restricted Area                1       1   
1630691   Left Side Center(LC) Above the Break 3              1       1   
1630698   Center(C)            Restricted Area                1       1   
          Left Side Center(LC) Above the Break 3              2       2   

                                                      MINUTES_REMAINING  \
PLAYER_ID SHOT_ZONE_AREA       SHOT_ZONE_BASIC                            


In [96]:
df

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0022100027,176,1626153,Delon Wright,1610612737,Atlanta Hawks,1,0,25,...,Center(C),Less Than 8 ft.,0,-4,6,1,1,20211023,CLE,ATL
1,Shot Chart Detail,0022100027,529,1626153,Delon Wright,1610612737,Atlanta Hawks,4,9,27,...,Center(C),Less Than 8 ft.,0,-2,-3,1,1,20211023,CLE,ATL
2,Shot Chart Detail,0022100043,580,1626153,Delon Wright,1610612737,Atlanta Hawks,4,4,57,...,Center(C),Less Than 8 ft.,4,23,43,1,1,20211025,ATL,DET
3,Shot Chart Detail,0022100059,448,1626153,Delon Wright,1610612737,Atlanta Hawks,3,2,28,...,Center(C),Less Than 8 ft.,7,65,44,1,1,20211027,NOP,ATL
4,Shot Chart Detail,0022100059,469,1626153,Delon Wright,1610612737,Atlanta Hawks,3,0,22,...,Right Side Center(RC),24+ ft.,25,157,197,1,1,20211027,NOP,ATL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51,Shot Chart Detail,0022100353,449,1629023,P.J. Washington,1610612766,Charlotte Hornets,3,1,46,...,Left Side Center(LC),24+ ft.,25,-163,194,1,1,20211206,CHA,PHI
52,Shot Chart Detail,0022100353,456,1629023,P.J. Washington,1610612766,Charlotte Hornets,3,0,41,...,Left Side Center(LC),24+ ft.,25,-131,219,1,1,20211206,CHA,PHI
53,Shot Chart Detail,0022100353,475,1629023,P.J. Washington,1610612766,Charlotte Hornets,4,11,51,...,Center(C),Less Than 8 ft.,0,6,6,1,1,20211206,CHA,PHI
54,Shot Chart Detail,0022100366,586,1629023,P.J. Washington,1610612766,Charlotte Hornets,4,3,25,...,Center(C),24+ ft.,25,-29,253,1,1,20211208,CHA,PHI
